<a href="https://colab.research.google.com/github/piegez/ai_generated_test/blob/main/rede_deteccao_facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people

# Carregando o dataset LFW
print("Carregando o dataset...")
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# Obtendo os dados e labels
X = lfw_people.images
y = lfw_people.target
target_names = lfw_people.target_names

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Pré-processando as imagens
X_train = np.repeat(X_train[..., np.newaxis], 3, -1)  # Convertendo para 3 canais
X_test = np.repeat(X_test[..., np.newaxis], 3, -1)
X_train = tf.image.resize(X_train, (224, 224)).numpy()
X_test = tf.image.resize(X_test, (224, 224)).numpy()
X_train = X_train / 255.0
X_test = X_test / 255.0

# Criando o modelo
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(len(target_names), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Compilando o modelo
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Treinando o modelo
print("Iniciando treinamento...")
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

# Plotando o histórico de treinamento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia (treino)')
plt.plot(history.history['val_accuracy'], label='Acurácia (validação)')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda (treino)')
plt.plot(history.history['val_loss'], label='Perda (validação)')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

# Avaliando o modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {test_acc:.4f}")

# Função para detectar e reconhecer faces
def detect_and_recognize_faces(image):
    # Pré-processamento da imagem
    image = tf.image.resize(image, (224, 224))
    image = tf.expand_dims(image, 0)

    # Fazendo a predição
    prediction = model.predict(image)
    person_id = tf.argmax(prediction[0]).numpy()
    confidence = tf.reduce_max(prediction[0]).numpy()

    return person_id, confidence

# Testando o modelo em algumas imagens do conjunto de teste
print("\nTestando o modelo em algumas imagens...")
for i in range(5):
    idx = np.random.randint(len(X_test))
    image = X_test[idx]
    true_label = y_test[idx]

    person_id, confidence = detect_and_recognize_faces(image)

    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(f'Predição: {target_names[person_id]}\nConfiança: {confidence:.2f}\nLabel Real: {target_names[true_label]}')
    plt.axis('off')
    plt.show()

Carregando o dataset...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Iniciando treinamento...
Epoch 1/10
11/25 ━━━━━━━━━━━━━━━━━━━━ 1:38 7s/step - accuracy: 0.3068 - loss: 1.7665